In [1]:
import import_ipynb # ipynb load 위한 모듈
import config
from processingdata import get_modified_data
from DeepFM import DeepFM

import numpy as np
import pandas as pd
from time import perf_counter
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import BinaryAccuracy, AUC

importing Jupyter notebook from config.ipynb
importing Jupyter notebook from processingdata.ipynb
importing Jupyter notebook from DeepFM.ipynb
importing Jupyter notebook from FMlayer.ipynb


In [2]:



def get_data():
    '''file = pd.read_csv('data/adult_csv.csv', header=None)
    file = file.dropna()
    file = file.drop(file.index[0],axis = 0)
    file = file.reset_index(drop = True)
    X = file.loc[:, 0:13]
    Y = file.loc[:, 14].map({'<=50K': 0, '>50K': 1})'''
    col = ["a","b","c","d","e","f","g"]

    da = [[1,2,3,"q","w","e",1],[4,5,6,"r","t","y",2],[7,8,9,"u","i","o",1],[1,2,3,"q","w","e",1]]

    f = pd.DataFrame(da,columns=col)
    X = f.drop('g',axis = 1)
    Y = f[['g']]
    
    X.columns = config.ALL_FIELDS
    
    field_dict, field_index, X_modified = \
        get_modified_data(X, config.ALL_FIELDS, config.CONT_FIELDS, config.CAT_FIELDS, False)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_modified, Y, test_size=0.2)  
    train_ds = tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(X_train.values, tf.float32), 
            tf.cast(Y_train, tf.float32)
        )
    ).shuffle(30000).batch(config.BATCH_SIZE)
    
    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test.values, tf.float32), 
         tf.cast(Y_test, tf.float32))
    ).shuffle(10000).batch(config.BATCH_SIZE)
    
    return train_ds, test_ds, field_dict, field_index


# Batch 단위 학습
def train_on_batch(model, optimizer, acc, auc, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False, y_true=targets, y_pred=y_pred)

    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용함
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # accuracy & auc
    acc.update_state(targets, y_pred)
    auc.update_state(targets, y_pred)

    return loss


# 반복 학습 함수
def train(epochs):
    train_ds, test_ds, field_dict, field_index = get_data()
    model = DeepFM(embedding_size=config.EMBEDDING_SIZE, num_feature=len(field_index),
                   num_field=len(field_dict), field_index=field_index)
    
   
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    print("Start Training: Batch Size: {}, Embedding Size: {}".format(config.BATCH_SIZE, config.EMBEDDING_SIZE))
    start = perf_counter()
    for i in range(epochs):
        acc = BinaryAccuracy(threshold=0.8)
        auc = AUC()
        loss_history = []

        for x, y in train_ds:
            loss = train_on_batch(model, optimizer, acc, auc, x, y)
            loss_history.append(loss)
            break

        print("Epoch {:03d}: 누적 Loss: {:.4f}, Acc: {:.4f}, AUC: {:.4f}".format(
            i, np.mean(loss_history), acc.result().numpy(), auc.result().numpy()))

    test_acc = BinaryAccuracy(threshold=0.5)
    test_auc = AUC()
    for x, y in test_ds:
        y_pred = model(x)
        test_acc.update_state(y, y_pred)
        test_auc.update_state(y, y_pred)

    print("테스트 ACC: {:.4f}, AUC: {:.4f}".format(test_acc.result().numpy(), test_auc.result().numpy()))
    print("Batch Size: {}, Embedding Size: {}".format(config.BATCH_SIZE, config.EMBEDDING_SIZE))
    print("걸린 시간: {:.3f}".format(perf_counter() - start))

In [3]:
train(1)

Data Prepared...
X shape: (4, 12)
# of Feature: 12
# of Field: 6
Start Training: Batch Size: 10, Embedding Size: 5
w  <tf.Variable 'w:0' shape=(12,) dtype=float32, numpy=
array([-1.4271162 ,  0.03568753,  1.669366  , -0.2503074 , -0.00237626,
       -1.5208621 ,  0.02526214, -0.73632413, -1.3800374 ,  0.9224091 ,
        1.8481983 , -0.03804522], dtype=float32)> 

v <tf.Variable 'V:0' shape=(6, 5) dtype=float32, numpy=
array([[ 0.00620536, -0.00989836, -0.014594  , -0.0077437 ,  0.00249628],
       [ 0.02184366, -0.00656004, -0.00520414,  0.00122091,  0.02219893],
       [ 0.00795335,  0.00110312,  0.01833142, -0.01320968,  0.00139492],
       [ 0.00442354,  0.00085361, -0.01419134,  0.00263469,  0.00014774],
       [ 0.00661631,  0.00592361, -0.00502302,  0.00200117,  0.00488387],
       [-0.0118293 , -0.00300177, -0.01880535,  0.01335137, -0.0168402 ]],
      dtype=float32)> 

input tf.Tensor(
[[4. 5. 6. 0. 1. 0. 0. 1. 0. 0. 0. 1.]
 [1. 2. 3. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [7. 8. 9. 0.

In [22]:
a = [
    [
        [1,2,3],[1,2,3]
    ],
    [
        [4,5,6],[4,5,6]
    ],
    [
        [6,6,7],[5,6,7]
]]

In [29]:
tf.square(tf.reduce_sum(tf.Variable(a),axis = [1,2]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 144,  900, 1369])>